##Pertemuan 3

In [ ]:
#install library mlxtend
!pip install mlxtend

In [1]:
import pandas as pd
import numpy as np
# import apriori from library mlxtend
from mlxtend.frequent_patterns import apriori, association_rules
import warnings
warnings.filterwarnings("ignore")

#Dataset
* Membuat dataset transaksi sebagai dataframe
* cek dataset transaksi

In [4]:
#membut data tabular yang terdiri dari data transaksi
df=pd.DataFrame(
    {
        "ID transaksi": pd.Series([1,2,3,4,5]),
        "item": pd.Series([["susu","roti","biskuit","mentega"],\
                          ["susu","biskuit","gula","kopi"],\
                           ["gula","kopi",'susu'],["kopi","biskuit","susu","gula"],\
                           ["mentega","roti","kopi"]])
    }
)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ID transaksi  5 non-null      int64 
 1   item          5 non-null      object
dtypes: int64(1), object(1)
memory usage: 208.0+ bytes


In [6]:
# Menampilkan Table Transaksi
df

,ID transaksi,item
0,1,"[susu, roti, biskuit, mentega]"
1,2,"[susu, biskuit, gula, kopi]"
2,3,"[gula, kopi, susu]"
3,4,"[kopi, biskuit, susu, gula]"
4,5,"[mentega, roti, kopi]"


# Encoding data transaksi

In [7]:
# Menampilkan Table Item
df = df.explode('item')
df

,ID transaksi,item
0,1,susu
0,1,roti
0,1,biskuit
0,1,mentega
1,2,susu
1,2,biskuit
1,2,gula
1,2,kopi
2,3,gula
2,3,kopi


In [14]:
basket = (df.groupby(['ID Transaksi', 'Item'])['Item'].count()\
                                      .unstack().reset_index().fillna(0)\
                                      .set_index('ID Transaksi'))
basket.head()

KeyError: 'ID Transaksi'

# Algoritma Apriori
* algoritma apriori ditunjukan untuk mengidenifikasi item-item yang sering muncul bersama dalam suatu transaksi atau kumpulan data.
* Algoritma Apriori sangat umum digunakan dalam analisis keranjang belanja, sistem rekomendasi, dan pemahaman pola konsumen
* Eksekusi algoritma apriori menggunakan library mlextend dengan fungsi apriori
* langkah lagkah umum apriori: mengeliminasi itemset berdasarkan minimum suport(threshold), menghitung confidence sejumlah perulangan tertentu, menentukan cross product atau kombinasi item-item yang muncul dalam satu transaksi

In [8]:
# Total per item
df_encode = df.groupby('item')["item"].count().reset_index(name="Count")
df_encode

,item,Count
0,biskuit,3
1,gula,3
2,kopi,4
3,mentega,2
4,roti,2
5,susu,4


# Apriori Tahap 1
Eliminasi itemset berdasarkan minimum support dan lakukan iterasi

In [ ]:
# @title
# # set minimum support 60%
# frequent_itemsets = apriori(basket, min_support=0.6, use_colnames=True)
# frequent_itemsets

In [ ]:
# @title
# rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.6)
# rules[["antecedents", "consequents", "support", "confidence"]]

# iterasi I
* Hitung support setiap item
* Asumsikan minimum support 3 atau 60% ( muncul 3x dari 5 transaksi)
* filter item yang tidak memenuhi minimum support


In [9]:
def create_support(x):
  # x jumlah transaksi per item
  N = 5 #jumlah semua transaksi
  y = (x/N)*100 #hitung persentase
  return y

In [10]:
df_i1 = df_encode.copy() #copy dataset df_encode ke df_i1 (df iterasi 1)
df_i1["Support"] = df_i1["Count"].apply(lambda x: create_support(x)) #apply fungsi ke df_i1 dengan tambahan kolom baru
df_i1

,item,Count,Support
0,biskuit,3,60.0
1,gula,3,60.0
2,kopi,4,80.0
3,mentega,2,40.0
4,roti,2,40.0
5,susu,4,80.0


In [11]:
df_i1 = df_i1[df_i1['Support'] >= 60] #filter data yang tidak memenuhi 60%
df_i1

,item,Count,Support
0,biskuit,3,60.0
1,gula,3,60.0
2,kopi,4,80.0
5,susu,4,80.0


# Iterasi II
* Kombinasi setiap item menjadi 2-itemset, hiraukan untuk item yang sudah difilter (roti, mentega)
* lalu filter kembali untuk 2-itemset yang tidak memenuhi minimum support

In [ ]:
# cek supportnya
filtered = ["Roti", "Mentega"]
df_filtered = df[~df["Item"].isin(filtered)]
df_encode_2 = df_filtered.groupby("Item")["Item"].count().reset_index(name="Count")
df_i2 = df_encode_2.copy()
df_i2["Support"] = df_i2["Count"].apply(lambda x: create_support(x))
df_i2

In [ ]:
df_i2 = df_i2[df_i2['Support'] >= 60] #filter data yang tidak memenuhi 60%
df_i2

#Apriori Tahap 2
Perhitungan confidence

# Kombinasi
Cross product atau kombinasi

In [ ]:
# cek kombinasi yang dihasilkan
basket_2 = (df_filtered.groupby(['ID Transaksi', 'Item'])['Item'].count()\
                                      .unstack().reset_index().fillna(0)\
                                      .set_index('ID Transaksi'))
frequent_itemsets_2 = apriori(basket_2, min_support=0.6, use_colnames=True)
frequent_itemsets_2

In [ ]:
rules = association_rules(frequent_itemsets_2, metric="confidence", min_threshold=0.6)
rules[["antecedents", "consequents", "support", "confidence"]]